# Shors Algorithm

## Task 1: A classical implementation

Initially, we will implement the framework for the 

First, we need to check special cases for Shor's Algorithm

### 1.1 IsEven

In [1]:
%kata IsEven_Test

operation IsEven(N : Int) : Bool {
    return (N%2==0);
}

Starting IsEven_Test...


Success!

### 1.2 IsPrime

In [2]:
%kata IsPrime_Test

open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;

operation IsPrime(N : Int) : Bool {
    if (N <= 3) {
        return (N > 1);
    }
    if (N%2 == 0) or (N%3 == 0) {
        return false;
    }
    let sqrtN = Truncate(Sqrt(IntAsDouble(N)));
    for i in 5..6..sqrtN {
        if (N%i == 0) or (N%(i+2) == 0) {
            return false;
        }

    }
    return true;
}

Starting IsPrime_Test...


Success!

GCD operation

In [3]:
%kata GreatestCommonDivisor_Test

open Microsoft.Quantum.Math;
operation GreatestCommonDivisor (a : Int, N : Int) : Int {
    return GreatestCommonDivisorI(a,N);
}

Starting GreatestCommonDivisor_Test...


Success!

With these two tests, we can confirm that N is a product of two distinct prime numbers, so $N = pq$.
// Explain order finding

### 1.3 Classical order finding

In [4]:
operation FindOrderClassicalHelper(a : BigInt, N: BigInt) : Int {
    mutable power = 0;
    repeat {
        set power += 1;
    } until (a^power % N == 1L);
    return power;
}

FindOrderClassicalHelper

In [5]:
%kata FindOrderClassical_Test

open Microsoft.Quantum.Convert;

operation FindOrderClassical(a : Int, N : Int) : Int{
    return FindOrderClassicalHelper(IntAsBigInt(a),IntAsBigInt(N));
}

Starting FindOrderClassical_Test...


Success!

### 1.4 Generate random number

In [6]:
%kata GenerateRandomNumber_Test

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Math;

operation GenerateRandomNumber(N : Int) : Int {
    use register = Qubit[BitSizeI(N-2)];
    mutable result = 0;
    repeat {
        ApplyToEachCA(H,register);
        set result = MeasureInteger(LittleEndian(register));
    } until ((result + 2) < N);
    return result+2;
}

Starting GenerateRandomNumber_Test...


Success!

### 1.5 General Case

In [7]:
%kata GeneralCase_Test

operation GeneralCase(OrderFinder : ((Int, Int)=>Int), N : Int) : (Int, Int) {
    mutable result = 0;
    repeat {
        let a = GenerateRandomNumber(N);
        let gcd = GreatestCommonDivisor(a,N);
        if (gcd > 1) {
            return (gcd, N/gcd);
        }
        let r = OrderFinder(a,N);
        if (IsEven(r)) {
            let x = (a^(r/2) - 1) % N;
            let gcdX = GreatestCommonDivisor(x,N);
            if (gcdX > 1) {
                set result = gcdX;
            }
        }
    } until (result != 0);
    return (result, N/result);
}

Starting GeneralCase_Test...


Success!

### 1.6 Full Shor's Implimentation

In [8]:
operation ShorsAlgorithm(OrderFinder : (Int, Int)=> Int, N : Int) : (Int, Int) {
    if IsEven(N) {
        return (2, N/2);
    }
    if IsPrime(N) {
        fail IntAsString(N) + " is prime, so doesn't have factors.";
    }
    return GeneralCase(OrderFinder, N);
}

ShorsAlgorithm

### 1.7 Classical Shor's Implimentation

In [9]:
%kata ShorsAlgorithmClassical_Test

operation ShorsAlgorithmClassical(N : Int) : (Int, Int) {
    return ShorsAlgorithm(FindOrderClassical, N);
}

Starting ShorsAlgorithmClassical_Test...


Success!

In [10]:
%simulate ShorsAlgorithmClassical N=15

(5, 3)

## Part 2: Quantum Implementation of Order Finding
### 2.1 Oracle

In [11]:
%kata OrderFindingOracle_Test

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Math;

operation OrderFindingOracle(a : Int, N : Int, power : Int, target : Qubit[]) : Unit is Adj+Ctl {
    MultiplyByModularInteger(ExpModI(a, power, N), N, LittleEndian(target));
}

Starting OrderFindingOracle_Test...


Success!

In [12]:
%kata PrepareEigenstate_Test

operation PrepareEigenstate(eigenstate : Qubit[]) : Unit is Adj+Ctl {
    X(eigenstate[0]);
}

Starting PrepareEigenstate_Test...


Success!

In [13]:
%kata ApplyQuantumPhaseEstimation_Test

open Microsoft.Quantum.Oracles;
open Microsoft.Quantum.Characterization;

operation ApplyQuantumPhaseEstimation(oracle : DiscreteOracle, eigenstate : Qubit[], result : Qubit[]) : Int {
    let resultBELE = LittleEndianAsBigEndian(LittleEndian(result));
    QuantumPhaseEstimation(oracle,eigenstate,resultBELE);
    return MeasureInteger(LittleEndian(result));
}

Starting ApplyQuantumPhaseEstimation_Test...


Success!

In [14]:
%kata PhaseResultToPeriod_Test

open Microsoft.Quantum.Math;

operation PhaseResultToPeriod(phaseResult : Int, bitsPrecision : Int, N : Int) : Int {
    let fractionResult = Fraction(phaseResult,2^bitsPrecision);
    let simplifiedFraction = ContinuedFractionConvergentI(fractionResult, N);
    let (numerator, period) = simplifiedFraction!;
    return AbsI(period);
}

Starting PhaseResultToPeriod_Test...


Success!

In [15]:
%kata FindOrderQuantum_Test

open Microsoft.Quantum.Diagnostics;

operation FindOrderQuantum(a : Int, N : Int) : Int {
    mutable result = 1;
    let bitsize = BitSizeI(N);
    let bitsPrecision = 2 * bitsize + 1; // decide on precision?
    
    repeat {        
        // setup register holding 'eigenstate' of |1>
        use eigenstate = Qubit[bitsize];
        PrepareEigenstate(eigenstate);
        
        // setup register to contain QFE result
        use phaseResult = Qubit[bitsPrecision];
        
        // measure the result from QPE
        let oracle = DiscreteOracle(OrderFindingOracle(a,N,_,_));
        //QuantumPhaseEstimation(oracle, eigenstate, LittleEndianAsBigEndian(phaseResultLE));
        let phaseResultI = ApplyQuantumPhaseEstimation(oracle, eigenstate, phaseResult);
        
        ResetAll(eigenstate);
        
        
        // calculate the period based of the phase result
        let period = PhaseResultToPeriod(phaseResultI,bitsPrecision,N);
        
        // deal with a zero return value
        if (period == 0) { set result = 1; }
        // account for sub factors
        else { set result = (period * result) / GreatestCommonDivisorI(result, period); }
        
    }
    until (ExpModI(a,result,N) == 1);
    return result;
}

Starting FindOrderQuantum_Test...


Success!

In [16]:
%kata ShorsAlgorithmQuantum_Test

operation ShorsAlgorithmQuantum(N : Int) : (Int, Int) {
    return ShorsAlgorithm(FindOrderQuantum,N);
}

Starting ShorsAlgorithmQuantum_Test...


Success!

In [ ]:
%simulate ShorsAlgorithmQuantum N=15